In [5]:
#shell of notebook

#Company Report Imports
import pandas as pd

import plotly.graph_objects as go

import networkx as nx
import ast
%matplotlib inline  
#from networkx.drawing.nx_agraph import write_dot, graphviz_layout
import matplotlib.pyplot as plt

#Company Suggestions Imports
import re
import sys
from IPython.display import display, HTML

#Display Dropdown Imports
import ipywidgets as widgets
from IPython.display import display
import functools

#Display Button Imports:
import asyncio



# 0. Core/Background Functions

### Display Dropdown

In [ ]:
def display_drop_down(df_column):
    """
    Passes in a Dataframe column to set the dropdown bar. Use in tandem with button code
    to create clickable button to direct users to different functions
    """
    dd = widgets.Dropdown(options= 
          df_column.unique(),
    value=df_column.iloc[0],
    description='Type:',
    disabled=False,
    )

    return dd

dd = display_drop_down(sic_sectors_codes['sector_group_name']) #set this to the column you like to define the code 


### Button Code

In [ ]:
slider = IntSlider()

async def f():
    for i in range(10):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

### Weighted Results

In [ ]:
#### weighting for search results

def weighted_result(x, sector):
    #use zone scoring to compute weight of query result    
    child_weight = .5
    sector_weight = .3
    company_weight = .2
    
    #returns positive if there are any children (favors multinational companies)
    child = (0,1)[x['num_children'] > 0]
    #returns positive if the company is an INC or CORP datatype (More likely to be a US multinational)
    company = (0,1)[x['company_type'] == 'INC' or x['company_type'] == 'CORP' or x['company_type'] == NaN]
    
    #attempts to put sector(if given) into related companies (replace with dropdown if possible)
    try:
        sect = (0,1)[sector.lower() in x['sector_name'].lower()]
    except:
        sect = 0

    return child_weight * child + sector_weight * sect + company_weight * company


### Unique Non-Nan (Need to define Location first)+  List Parse

In [ ]:
def unique_non_nan(elem):
    return [a for a in list(pd.unique(elem))]
#maybe shouldn't just take the first one 
location_filtered = location[location['max_year'] > 2018]
location_aggregations = {'city': "first",
                         'state': "first",
                         'country_code':  "first"}
location_agg = location_filtered.groupby('cw_id').agg(location_aggregations)
#location_df = location_filtered[['cw_id', 'city', 'state', 'country_code']]

In [2]:
#required to read the list column from a file 
def list_parse(s):
    strings = str(s).strip("[]").split(",")
    ls = []
    for i in strings:
        try:
            ls.append(int(float(i.strip())))
        except (ValueError):
            ""
    return ls

# A. Start Page

Welcome to the Multinational 

### Workflow
1. Users choose what kind of search they want


### Choices

- Choose a Company (Go to -> __C. Company Query__)

- Choose a Sector (Go to -> __B. Search by Sector__)

- (Optional) Choose from top multinational companies


##### Disclaimer
Only includes US companies

In [6]:
def AStart():
    '''Display start page + options'''
    #B = Sector
    #C = Company
    while True:
        choice = input("Please select an option: ")
        if choice == "B":
            BSector()
        elif choice == "C":
            CCompany()
        else:
            print("sorry I didn't get that, please try again")

## B. Search by Sector

### Workflow
1. Users select Sector Group (dropdown)
1b. Users select Sector (dropdown)
1c. Users select Industry (dropdown)

2. Top Players Results displayed
3. Users select company to view report (dropdown) (Go to -> __D. Company Report Page__)
3b. Users go back (Go to -> __A. Start Page__)

### Features
- Dropdown Widgets: Sector selection
- Top Players Results
    - Groupby: Industries
    - Groupby: Top Parent IDs
- Dropdown Widgets: Company selection



In [7]:
def BSector():
    '''display sectors options'''
    display_drop_down(Sector_Groups)
    display_drop_down(Sectors)
    display_drop_down(Industries)
    SIC_choice = "Sector/industry"
    
    search_by_sector(SIC_choice)
    #displays top 25 results
    
    company_choice = display_drop_down(Companies)
    display_company_report = company_choice

    

In [8]:
# rachel will start
def display_drop_down(options_list):
    """
    displays a drop down for the items in options list, collects choice from the user and returns a string with
    their choice
    
    Requirements: Sector_group_name, Sector_name, Industry_name
    """
    choice = "Berkshire Hathway"
    return choice

In [9]:
#prakar
def search_by_sector(sector_name):
    """
    returns list of a parent companies in that sub sector, ranked by subsidiaries
    """
    return []

## C. Search by Company

### Workflow
1. Users search by Company Name
1b. Users select Sector (dropdown)
1c. Users select Industry (dropdown)

2. Company Results displayed
3. Users select company to view report (dropdown) (Go to -> __D. Company Report Page__)
3b. Users go back (Go to -> __A. Start Page__)

### Features
- Dropdown Widgets: Sector selection
- Weighted Zone Search
    - Number of Children
    - Company Type
    - Sector Type
- Dropdown Widgets: Company selection

In [10]:
def CCompany():
    company_suggestions()
    

In [13]:
# Max is going to write this to start with
#company_joined2['weighted_result'] = 'default'
import re
import sys
from IPython.display import display, HTML
def company_suggestions():
    '''
    if name isn't clean, then search name+wildcard to return list of companies, ranked by number of children.
    can remove
    '''
    
    '''
    first test if name entered finds something in the database
    '''
    
    print('Please input a company you would like to search for')
    print('Tips for searching:')
    print('-You can enter a partial number')
    company = input('> ').upper()
    
    sector = input('What sector are you looking for: ')
    #company_joined2.columns
    df = pd.DataFrame(company_joined2, columns = ['cw_id', 'company_name', 'num_children', 'sector_name', 'weighted_result', 'company_type'])
#     df.head()
    #display_company_report(company) #navigates to company report if selected
    df = df[ df['company_name'].str.match(company) ]
    if df.size == 0:
        print("I'm sorry, I wasn't able to find a company with those attributes. Would you like to do next:")
        print("1) Search for another company?")
        print("2) Go Back")
        company = input('> ').upper()

    df['weighted_result'] = df[['num_children','sector_name','company_type']].apply(lambda x: weighted_result(x,sector), axis=1)
    df = df.sort_values(by=['weighted_result'], ascending = False)
    display(df)
    x = 0
    while x < 3:
        choice = None
        if df.size == 0:
            print("I'm sorry, I wasn't able to find a company with those attributes. Would you like to do next:")
            print("1) Search for another company?")
            print("2) Go Back")
            choice = int(input("> "))
        elif df.size != 1:
            print('which company would you like to select (cw_id):')   
            cw = int(input('>'))
            company = df.loc[df['cw_id'] == cw, ['company_name', 'child_cw_id_list'] ]
        else:
            company = df.iloc[0]
            print(company)

        if choice != None:
            print('What would you like to do now:')
            print('1) Search for another Company')
            print('2) Go Back')
            print('3) Get Company Report')
            choice = int(input("> "))
        else:
            if choice == 1:
                company_suggestions()
            elif choice == 2:
                test = ''
                #go back to previous page
            elif choice == 3:
                display_breakdown(company, cw)
            else:
                print('Sorry I didnt get that, please select another choice')
        x+=1
    print('Sorry you have failed too many times. Please try again')
    sys.exit()
    
    return None#ranked list of company suggestions
#company_suggestions()

Please input a company you would like to search for
Tips for searching:
-You can enter a partial number


KeyboardInterrupt: 

## D. Company Report Page

### Workflow

1. User is shown page
2. User select another company to view report (dropdown) (Go to -> __D. Company Report Page__)
2b. Users go back (Go to -> __A. Start Page__)


### Features

#### Parent Company
- Relationship Map
- Company Breakdown (% of Company Industries)
    - Groupby: Top Parent ID
    - Groupby: Sector
    - Top Players Ranking
        - Top Players Results
- Foreign/Domestic (% of Domestic vs. International Companies)
    - Groupby: Location
    - Groupby: State
- Dropdown Widgets: Company selection


#### Subsidiary
- Similar Companies
    - Groupby: Top Parent ID
    - Groupby: Sector
- Dropdown Widgets: Company selection

In [14]:
#rachel with start
def display_company_report(clean_company_name):
    """
    given a clean company name, show a display report of the company 
    
    SIC 
    LOC
    F/D % percent of companies that are abroad vs. domestic
    
    
    Requirements: Corpwatch Company data, company relationship data

    """
    display_breakdown(clean_company_name)
    display_hierarchy(clean_company_name)
    
    company = input('')
    display_company_report(company)
    #display something
    #return the report

In [15]:
def display_breakdown(clean_company_name):
    '''
    uses logic to determine if it should display (e.g. is it a parent)
    
    if so, shows Pie chart of sector breakdown
    
    shows list of sectors they are in
    '''
    return []

In [16]:
def display_hierarchy(clean_company_name):
    '''
    uses logic to determine if it should display (e.g. does it have at least a parent or a child)
    
    if so, show table of parent (if applicable)
    show table of children (if applicable)
    '''
    return []

In [17]:
if result == 'sector':
    #show sector group
    
    sector_company_list = search_by_sector(sector_name): #returns all companies in sector
    company = display_dropdown(sector_company_list) #displays companies with subsidiaries 
    display_company_report(company) #navigates to company report if selected
    #show sector name
    #show industry name
    

SyntaxError: invalid syntax (<ipython-input-17-678471962d70>, line 4)